# Decision Tree

## git push

## 데이터 전처리

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn import preprocessing

datapath = "~/ML2021/data/diagnosis.data"

original_data = pd.read_csv(datapath, sep = "\t", encoding = "utf-16", names = ["temperature","nausea","lumbar_pain","urine_pushing","micturition_pains","burning_of_urethra","inflamation","nephritis"])

1. 칼럼 temperature 에 , 를 . 으로 수정이 필요함.
2. 다른 칼럼 데이터에 no, yes를 0,1로 수정이 필요함.

In [2]:
original_data.head()

,temperature,nausea,lumbar_pain,urine_pushing,micturition_pains,burning_of_urethra,inflamation,nephritis
0,"35,5",no,yes,no,no,no,no,no
1,"35,9",no,no,yes,yes,yes,yes,no
2,"35,9",no,yes,no,no,no,no,no
3,"36,0",no,no,yes,yes,yes,yes,no
4,"36,0",no,yes,no,no,no,no,no


apply(lambda)를 사용하여 , 를 . 으로 수정

In [3]:
original_data['temperature'] = original_data['temperature'].apply(lambda x: x.replace(',','.'))

original_data 에서 dicted_data로 수정하며 preprocessing.LabelEncoder()를 <br>
이용하여 0과 1로 데이터 변환

In [4]:
dicted_data = original_data.copy()
tmp = ['nausea','lumbar_pain','urine_pushing','micturition_pains','burning_of_urethra','inflamation','nephritis']
les = {}

for x in tmp:
    les[x] = preprocessing.LabelEncoder()
    dicted_data[x] = les[x].fit_transform(original_data[x])
dicted_data.head()

,temperature,nausea,lumbar_pain,urine_pushing,micturition_pains,burning_of_urethra,inflamation,nephritis
0,35.5,0,1,0,0,0,0,0
1,35.9,0,0,1,1,1,1,0
2,35.9,0,1,0,0,0,0,0
3,36.0,0,0,1,1,1,1,0
4,36.0,0,1,0,0,0,0,0


## Decision Model 생성

X : temperature 만 feature로 지정
Y : 예측할 label를 'inflammation' 으로 지정

In [9]:
x = dicted_data['temperature']
y = dicted_data['inflamation']
model = DecisionTreeClassifier(min_samples_split = 20, random_state = 99)

In [6]:
y.head()

0    0
1    1
2    0
3    1
4    0
Name: inflamation, dtype: int64

In [7]:
x.head()

0    35.5
1    35.9
2    35.9
3    36.0
4    36.0
Name: temperature, dtype: object

In [10]:
model.fit(x,y)

ValueError: Expected 2D array, got 1D array instead:
array=[35.5 35.9 35.9 36.  36.  36.  36.2 36.2 36.3 36.6 36.6 36.6 36.6 36.7
 36.7 36.7 36.8 36.8 36.9 36.9 37.  37.  37.  37.  37.  37.  37.  37.
 37.1 37.1 37.1 37.2 37.2 37.2 37.3 37.3 37.3 37.4 37.4 37.5 37.5 37.5
 37.5 37.5 37.5 37.6 37.6 37.6 37.7 37.7 37.7 37.7 37.8 37.8 37.8 37.9
 37.9 37.9 37.9 37.9 38.  38.  38.1 38.3 38.5 38.7 38.9 39.  39.4 39.7
 40.  40.  40.  40.  40.  40.  40.  40.  40.1 40.2 40.2 40.2 40.3 40.4
 40.4 40.4 40.4 40.4 40.5 40.6 40.6 40.6 40.7 40.7 40.7 40.7 40.7 40.8
 40.9 40.9 40.9 41.  41.  41.  41.  41.1 41.1 41.1 41.1 41.1 41.2 41.2
 41.2 41.2 41.3 41.4 41.5 41.5 41.5 41.5].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.